In [1]:
import os
# os.environ['CUDA_VISIBLE_DEVICES'] = '-1'
from PIL import Image
from keras.models import load_model
from keras_facenet import FaceNet
import numpy as np
from mtcnn import MTCNN
import cv2
import pickle
from IPython.display import display
import pandas as pd

In [2]:
from tensorflow.keras.preprocessing.image import img_to_array,array_to_img
embedder = FaceNet()
import mysql.connector

In [3]:
import tensorflow as tf
tf.config.list_physical_devices()

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'),
 PhysicalDevice(name='/physical_device:XLA_CPU:0', device_type='XLA_CPU'),
 PhysicalDevice(name='/physical_device:XLA_GPU:0', device_type='XLA_GPU'),
 PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [4]:
df = pd.read_excel("StudentListEmail_ccet_gmail.xlsx").fillna("example@domain.com")
df.head(5)

,First Name,Roll Number,Email Address [CCET],Email [Secondary]
0,Aarushi Sood,CO17301,example@domain.com,example@domain.com
1,Abhijeet Baruah,CO17302,example@domain.com,example@domain.com
2,Abhishek Kaushik,CO17304,example@domain.com,example@domain.com
3,Aboli,CO17305,example@domain.com,example@domain.com
4,Agampreet Kaur Walia,CO17307,example@domain.com,example@domain.com


In [5]:
#this script is just to automate the insertion of multiple bnde    
name = list(df['First Name '])
email = list(df['Email [Secondary]'])
ccet_email = list(df['Email Address [CCET]'])
roll= list(df['Roll Number'])

class_temp_path ="/home/zukayu/Eye_Attend/Eye_Attend_Final/classroom_224"

temp_totalEmbeddings = []
for faces in os.listdir(class_temp_path):
    face = Image.open(class_temp_path+os.sep+faces)
    face = img_to_array(face)
    face = np.expand_dims(face, axis=0)
    embedding = embedder.embeddings(face)
    temp_totalEmbeddings.append(embedding)

In [6]:
print(temp_totalEmbeddings[0].shape)
print(len(temp_totalEmbeddings))
# This means it is 45 x 512
print(len(roll))

(1, 512)
45
45


In [8]:
conn = mysql.connector.connect(
    host="localhost",
    user="root",
    password="",
    database="eyeattend"
    )
if conn.is_connected():
    print("Successfully connected")
    mycursor = conn.cursor()
    #sql = "INSERT INTO `embeddings`(`embedd`) VALUES (%s)"
    #"INSERT INTO table VALUES (%s, %s, %s)", (pickles)
    create_tab = "CREATE TABLE IF NOT EXISTS `batch_2018` ( `roll_no` varchar(10) NOT NULL, \
                 `name` varchar(50) NOT NULL,\
                 `branch` text NOT NULL,\
                 `batch` text NOT NULL,\
                 `email` varchar(50) NOT NULL,\
                 `ccet_email` varchar(50) NOT NULL,\
                 `photo_embedd` longblob NOT NULL,\
                 PRIMARY KEY (`roll_no`) ) "
    
    mycursor.execute(create_tab)
    conn.commit()
    
    for i in range(len(roll)):
        r = str(roll[i])
        naam = str(name[i])
        ema = str(email[i])
        ccet = str(ccet_email[i])
        embed = temp_totalEmbeddings[i]
        pickemb = embed.dumps()
        mycursor.execute("INSERT INTO batch_2018 VALUES (%s,%s,%s,%s,%s,%s,%s)",(r,naam,'cse','2018',ema,ccet,pickemb))
        conn.commit()
        # Closing the connection
    conn.close()
    
else:
    print('Error in connecting with database') 

Successfully connected
